# Template Model Classification - Dataset Capfiltered

In [9]:
#import semua library yg bakalan dipake di sini, biar rapih
#kalo kurang tambahin aja yaa wkwkwk

import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier


from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, precision_score, recall_score, f1_score, accuracy_score, auc, classification_report
from sklearn.metrics import ConfusionMatrixDisplay

from xgboost import XGBClassifier

In [10]:
df_train_capfiltered = pd.read_csv('processed_dataset/Train/processed_Train_capfiltered.csv')
df_test_capfiltered = pd.read_csv('processed_dataset/Test/processed_Test_capfiltered.csv')

In [11]:
df_train_capfiltered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Customer_care_calls         10500 non-null  float64
 1   Customer_rating             10500 non-null  float64
 2   Cost_of_the_Product         10500 non-null  float64
 3   Prior_purchases             10500 non-null  float64
 4   Discount_offered            10500 non-null  float64
 5   Weight_in_gms               10500 non-null  float64
 6   Gender_encoded              10500 non-null  int64  
 7   Product_importance_encoded  10500 non-null  int64  
 8   Warehouse_block_A           10500 non-null  int64  
 9   Warehouse_block_B           10500 non-null  int64  
 10  Warehouse_block_C           10500 non-null  int64  
 11  Warehouse_block_D           10500 non-null  int64  
 12  Warehouse_block_F           10500 non-null  int64  
 13  Mode_of_Shipment_Flight     105

In [12]:
df_test_capfiltered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Customer_care_calls         2200 non-null   float64
 1   Customer_rating             2200 non-null   float64
 2   Cost_of_the_Product         2200 non-null   float64
 3   Prior_purchases             2200 non-null   float64
 4   Discount_offered            2200 non-null   float64
 5   Weight_in_gms               2200 non-null   float64
 6   arrived_late                2200 non-null   int64  
 7   Gender_encoded              2200 non-null   int64  
 8   Product_importance_encoded  2200 non-null   int64  
 9   Warehouse_block_A           2200 non-null   int64  
 10  Warehouse_block_B           2200 non-null   int64  
 11  Warehouse_block_C           2200 non-null   int64  
 12  Warehouse_block_D           2200 non-null   int64  
 13  Warehouse_block_F           2200 

In [13]:
#split X dan Y 
#moga2 split train sama testnya bener ya KWKWKWKWK

X_train = df_train_capfiltered.drop('arrived_late', axis = 1)
y_train = df_train_capfiltered['arrived_late']
X_test = df_test_capfiltered.drop('arrived_late', axis = 1)
y_test = df_test_capfiltered['arrived_late']

In [14]:
def eval_classification(model):
    y_pred = model.predict(X_test)
    y_pred_train = model.predict(X_train)
    y_pred_proba = model.predict_proba(X_test)
    y_pred_proba_train = model.predict_proba(X_train)
    
    print("Accuracy (Test Set): %.2f" % accuracy_score(y_test, y_pred))
    print("Precision (Test Set): %.2f" % precision_score(y_test, y_pred))
    print("Recall (Test Set): %.2f" % recall_score(y_test, y_pred))
    print("F1-Score (Test Set): %.2f" % f1_score(y_test, y_pred))
    
    print("AUC (test-proba): %.2f" % roc_auc_score(y_test, y_pred_proba[:, 1]))
    print("AUC (train-proba): %.2f" % roc_auc_score(y_train, y_pred_proba_train[:, 1]))
#     print("AUC (test-label): %.2f" % roc_auc_score(y_test, y_pred))
#     print("AUC (train-label): %.2f" % roc_auc_score(y_train, y_pred_train))
    print('\n------------------------\n')
    print(classification_report(y_test, y_pred))

def show_feature_importance(model):
    feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
    ax = feat_importances.nlargest(25).plot(kind='barh', figsize=(10, 8))
    ax.invert_yaxis()

    plt.xlabel('score')
    plt.ylabel('feature')
    plt.title('feature importance score')

#feature importance untuk SVM
def feature_plot(classifier, feature_names, top_features=8):
    coef = classifier.coef_.ravel()
    top_positive_coefficients = np.argsort(coef)[-top_features:]
    top_negative_coefficients = np.argsort(coef)[:top_features]
    top_coefficients = np.hstack([top_negative_coefficients, top_positive_coefficients])
    plt.figure(figsize=(18, 7))
    colors = ['red' if c < 0 else 'blue' for c in coef[top_coefficients]]
    plt.bar(np.arange(2 * top_features), coef[top_coefficients], color=colors)
    feature_names = np.array(feature_names)
    plt.xticks(np.arange(0, 0 + 2 * top_features), feature_names[top_coefficients], rotation=45, ha='right')
    plt.show()
    
def show_best_hyperparameter(model):
    print(model.best_estimator_.get_params())

In [15]:
def draw_learning_curve(param_values):
    train_scores = []
    test_scores = []

    for c in param_values:
        model = LogisticRegression(penalty='l2', C=c)
        model.fit(X_train, y_train)

        # eval on train
        y_pred_train_proba = model.predict_proba(X_train)
        train_auc = roc_auc_score(y_train, y_pred_train_proba[:,1])
        train_scores.append(train_auc)

        # eval on test
        y_pred_proba = model.predict_proba(X_test)
        test_auc = roc_auc_score(y_test, y_pred_proba[:,1])
        test_scores.append(test_auc)

        print('param value: ' + str(c) + '; train: ' + str(train_auc) + '; test: '+ str(test_auc))

    plt.plot(C, train_scores, label='Train')
    plt.plot(C, test_scores, label='Test')
    plt.legend()
    plt.show()

# AdaBoost (Arsya) - Best Fit


In [8]:
ada = AdaBoostClassifier()
ada.fit(X_train,y_train)

y_pred = rf.predict(X_test)
eval_classification(ada)

NameError: name 'rf' is not defined

In [ ]:
show_feature_importance(ada)

# Logistic Regression (Edo) - Best Fit


In [ ]:
lr = LogisticRegression()
lr.fit(X_train,y_train)

y_pred = rf.predict(X_test)
eval_classification(lr)

In [ ]:
show_feature_importance(lr)

# Decision Tree

In [ ]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)

y_pred = dt.predict(X_test)
eval_classification(dt)

Semua hasil evaluasi memiliki nilai 1.0 hal yang sangat mustahil di dunia nyata, apakah ada kesalahan? Diperlukan tunning untuk mengurangi nilainya.

In [ ]:
show_feature_importance(dt)

# Random Forest

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
eval_classification(rf)

# Support Vector Classification (Agas)

In [ ]:
svc = SVC(probability=True, random_state=42)
svc.fit(X_train, y_train)

y_pred = svc.predict(X_test)
eval_classification(svc)

# K-Nearest Neighbor - Minta tolong Run

In [16]:
knn = KNeighborsClassifier()

knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)
eval_classification(knn)

Accuracy (Test Set): 0.63
Precision (Test Set): 0.74
Recall (Test Set): 0.58
F1-Score (Test Set): 0.65
AUC (test-proba): 0.70
AUC (train-proba): 0.89

------------------------

              precision    recall  f1-score   support

           0       0.53      0.70      0.60       887
           1       0.74      0.58      0.65      1313

    accuracy                           0.63      2200
   macro avg       0.64      0.64      0.63      2200
weighted avg       0.66      0.63      0.63      2200



In [ ]:
show_feature_importance(knn)

# XGBoost (Riyan)

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

y_pred = rf.predict(X_test)
eval_classification(xgb)

In [ ]:
show_feature_importance(xgb)